In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=30249ea1e5187efd69a25cf1b3a1164f68bc977a45cac8c33f63f71bde9e5ec1
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
import numpy as np
import pandas as pd
import torch
import json

# from datasets import Dataset
from functools import partial
# from seqeval.metrics import accuracy_score, classification_report
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer, Trainer, TrainingArguments
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification
from transformers import RobertaTokenizerFast
from tqdm import tqdm
from pathlib import Path
from transformers.tokenization_utils import PreTrainedTokenizerBase
from transformers.models.deberta_v2 import DebertaV2ForTokenClassification, DebertaV2TokenizerFast

In [ ]:
OUTPUT_DIR = "/content/output"
Path(OUTPUT_DIR).mkdir(exist_ok=True)


# **Constants**

In [ ]:

seed = 42
model_checkpoint = "FacebookAI/roberta-base"
model_train = '/content/output'
model_final = '/content/output'
max_length = 512
stride = 64

# hyperparameter
epochs = 5
learning_rate = 3e-5
warmup_ratio = 0.1
lr_scheduler_type='cosine'
weight_decay = 0.01
grad_steps = 2
batch_size = 8

# PII (NER) tags
labels = ["B-EMAIL", "B-ID_NUM", "B-NAME_STUDENT", "B-PHONE_NUM",
          "B-STREET_ADDRESS", "B-URL_PERSONAL", "B-USERNAME",
          "I-ID_NUM", "I-NAME_STUDENT", "I-PHONE_NUM",
          "I-STREET_ADDRESS","I-URL_PERSONAL","O"]
id2label = dict(enumerate(labels)) # integer label to BIO format label mapping
label2id = {v:k for k,v in id2label.items()} # BIO format label to integer label mapping
num_labels = len(labels) # number of PII (NER) tags


In [ ]:
import json
original_data = json.load(open('/content/drive/MyDrive/PII/train.json'))

external_data = json.load(open('/content/drive/MyDrive/PII/mpware_mixtral8x7b_v1.1-no-i-username.json'))


data = original_data + external_data
print("original:", len(original_data))
print("external:", len(external_data))

original: 6807
external: 2692


In [ ]:
import pandas as pd
df = pd.DataFrame(data)
df

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."
...,...,...,...,...,...
9494,72efe6ee9844b8c802098af0585397bd,Mind Mapping Tool for Idea Generation and Prob...,"[Mind, Mapping, Tool, for, Idea, Generation, a...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
9495,d2ceb11151ea660a3d20238e36c169f1,Mind Mapping Tool for Idea Generation and Prob...,"[Mind, Mapping, Tool, for, Idea, Generation, a...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
9496,21bce330d38baa5be1a7018e1bf82fe2,Mind Mapping Tools for Idea Generation and Pro...,"[Mind, Mapping, Tools, for, Idea, Generation, ...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
9497,0d876118e85ef8a4720a7f12783f7dc3,Mind Mapping Tool for Idea Generation and Prob...,"[Mind, Mapping, Tool, for, Idea, Generation, a...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [ ]:
from itertools import chain
from collections import Counter
labels_chained = chain.from_iterable(df['labels'])
label_counts = Counter(labels_chained)
label_counts

Counter({'O': 7076272,
         'B-NAME_STUDENT': 5979,
         'I-NAME_STUDENT': 6962,
         'B-URL_PERSONAL': 1348,
         'B-EMAIL': 909,
         'B-ID_NUM': 1111,
         'I-URL_PERSONAL': 1,
         'B-USERNAME': 847,
         'B-PHONE_NUM': 823,
         'I-PHONE_NUM': 2545,
         'B-STREET_ADDRESS': 801,
         'I-STREET_ADDRESS': 6239,
         'I-ID_NUM': 1241})

In [ ]:
all_labels = [
    'B-EMAIL', 'B-ID_NUM', 'B-NAME_STUDENT', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'B-URL_PERSONAL', 'B-USERNAME', 'I-ID_NUM', 'I-NAME_STUDENT', 'I-PHONE_NUM', 'I-STREET_ADDRESS', 'I-URL_PERSONAL', 'O'
]
id2label = {i: l for i, l in enumerate(all_labels)}
label2id = {v: k for k, v in id2label.items()}
target = [l for l in all_labels if l != "O"]

In [ ]:
# This function is almost used by all the participants in the competition
def tokenize(example, tokenizer, label2id, max_length):

    text = []
    labels = []

    for t, l, ws in zip(
        example["tokens"], example["provided_labels"], example["trailing_whitespace"]
    ):
        text.append(t)
        labels.extend([l] * len(t))

        if ws:
            text.append(" ")
            labels.append("O")

    tokenized = tokenizer("".join(text), return_offsets_mapping=True, max_length=max_length)

    labels = np.array(labels)

    text = "".join(text)
    token_labels = []

    for start_idx, end_idx in tokenized.offset_mapping:
        if start_idx == 0 and end_idx == 0:
            token_labels.append(label2id["O"])
            continue

        if text[start_idx].isspace():
            start_idx += 1

        token_labels.append(label2id[labels[start_idx]])

    length = len(tokenized.input_ids)

    return {**tokenized, "labels": token_labels, "length": length}


In [ ]:
def tokenize_and_align_labels(example, tokenizer, label2id, max_length, stride):
    tokenized_inputs = tokenizer(example["tokens"],
                                 truncation=True,
                                 max_length=512,
                                 stride=64,
                                 padding="max_length",
                                 is_split_into_words=True,
                                 return_overflowing_tokens=True)
    tokenized_overflow_mappings = tokenized_inputs.overflow_to_sample_mapping
    labels = []
    for idx, overflow_mapping_id in enumerate(tokenized_overflow_mappings):
        label_ids = []
        previous_word_idx = None
        word_ids = tokenized_inputs.word_ids(batch_index=idx)
        label = example["pii_labels"]
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
ds = Dataset.from_dict({
    "full_text": [x["full_text"] for x in data],
    "document": [str(x["document"]) for x in data],
    "tokens": [x["tokens"] for x in data],
    "trailing_whitespace": [x["trailing_whitespace"] for x in data],
    "pii_labels": [x["labels"] for x in data],
})

In [ ]:
ds = ds.train_test_split(test_size=0.2, seed=seed)
ds

DatasetDict({
    train: Dataset({
        features: ['full_text', 'document', 'tokens', 'trailing_whitespace', 'pii_labels'],
        num_rows: 7599
    })
    test: Dataset({
        features: ['full_text', 'document', 'tokens', 'trailing_whitespace', 'pii_labels'],
        num_rows: 1900
    })
})

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained(model_checkpoint, add_prefix_space = True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
x = ds['train'][7085]

for t,l in zip(x["tokens"], x["pii_labels"]):
    if l != "O":
        print((t,l))

print("*"*100)

for t, l in zip(tokenizer.convert_ids_to_tokens(x["input_ids"][0]), x["labels"][0]):
    if l != -100 and l != 12:
        print((t,id2label[l]))

('Katelyn', 'B-NAME_STUDENT')
("O'Connell", 'I-NAME_STUDENT')
('-', 'I-NAME_STUDENT')
('McLaughlin', 'I-NAME_STUDENT')
('Noemi22@yahoo.com', 'B-EMAIL')
('Estrella_Dare', 'B-USERNAME')
('823', 'B-ID_NUM')
('20', 'I-ID_NUM')
('2460', 'I-ID_NUM')
('+432', 'B-PHONE_NUM')
('-', 'I-PHONE_NUM')
('527', 'I-PHONE_NUM')
('572.8060', 'I-PHONE_NUM')
('https://medical-afternoon.com/Estrella_Dare', 'B-URL_PERSONAL')
('Lexus', 'B-STREET_ADDRESS')
('Bridge', 'I-STREET_ADDRESS')
('Apt', 'I-STREET_ADDRESS')
('.', 'I-STREET_ADDRESS')
('664', 'I-STREET_ADDRESS')
('Kirkland', 'I-STREET_ADDRESS')
(',', 'I-STREET_ADDRESS')
('French', 'I-STREET_ADDRESS')
('Guiana', 'I-STREET_ADDRESS')
('20830', 'I-STREET_ADDRESS')
****************************************************************************************************
('ĠK', 'B-NAME_STUDENT')
('ĠO', 'I-NAME_STUDENT')
('Ġ-', 'I-NAME_STUDENT')
('ĠMcL', 'I-NAME_STUDENT')
('ĠNo', 'B-EMAIL')
('ĠEst', 'B-USERNAME')
('Ġ8', 'B-ID_NUM')
('Ġ20', 'I-ID_NUM')
('Ġ24', 'I-ID_NU

In [ ]:
del x

In [ ]:
tokenized_ds = ds.remove_columns(['document', 'tokens', 'pii_labels', 'overflow_to_sample_mapping'])
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['full_text', 'trailing_whitespace', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 7599
    })
    test: Dataset({
        features: ['full_text', 'trailing_whitespace', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1900
    })
})

In [ ]:
def rearrange_tokenized_input(dataset):
    tokenized = {
        "input_ids": [],
        "labels": [],
        "attention_mask": [],
    }
    for i in tqdm(range(len(dataset))):
        doc = dataset[i]
        for j in range(len(doc["input_ids"])):
            tokenized["input_ids"].append(doc["input_ids"][j])
            tokenized["labels"].append(doc["labels"][j])
            tokenized["attention_mask"].append(doc["attention_mask"][j])
    return tokenized

In [ ]:
# rearrange tokenized data to model input format
tokenized_ds["train"] = Dataset.from_dict(rearrange_tokenized_input(tokenized_ds["train"]))
tokenized_ds["test"] = Dataset.from_dict(rearrange_tokenized_input(tokenized_ds["test"]))
tokenized_ds

100%|██████████| 1900/1900 [00:04<00:00, 438.33it/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 16540
    })
    test: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 4198
    })
})

In [ ]:
# from HuggingFace https://huggingface.co/learn/nlp-course/chapter7/2?fw=pt#metrics
def compute_metrics(eval_preds, id2label):
    predictions, labels = eval_preds
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    report = classification_report(y_true=true_labels, y_pred=true_predictions, output_dict=True)
    micro_avg = report.pop("micro avg")
    accuracy = accuracy_score(y_true=true_labels, y_pred=true_predictions)

    return {
        "precision": micro_avg["precision"],
        "recall": micro_avg["recall"],
        "f1": micro_avg["f1-score"],
        "accuracy": accuracy
    }

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

training_args = TrainingArguments(
    output_dir=model_train,
    num_train_epochs=epochs,
    learning_rate=learning_rate,
    lr_scheduler_type=lr_scheduler_type,
    warmup_ratio=warmup_ratio,
    weight_decay=weight_decay,
    gradient_accumulation_steps=grad_steps,
    per_device_train_batch_size=batch_size,
    seed=seed,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="f1", #Here I used f1 before realising that the required is f5.However we can basically estimate the f5 score if it's good or bad
    greater_is_better=True,
    load_best_model_at_end=True
)


data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=partial(compute_metrics, id2label=id2label)
)

trainer.train()
trainer.save_model(model_final)
tokenizer.save_pretrained(model_final)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at /content/drive/MyDrive/roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.001300,0.000571,0.958140,0.941812,0.949906,0.999849
2,0.000500,0.000553,0.957272,0.968412,0.962810,0.999869


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.001300,0.000571,0.958140,0.941812,0.949906,0.999849
2,0.000500,0.000553,0.957272,0.968412,0.962810,0.999869
3,0.000200,0.000459,0.976381,0.962178,0.969228,0.999899
4,0.000100,0.000418,0.972758,0.964672,0.968698,0.999900
5,0.000100,0.000447,0.971714,0.970906,0.971310,0.999903


('/content/output/tokenizer_config.json',
 '/content/output/special_tokens_map.json',
 '/content/output/vocab.json',
 '/content/output/merges.txt',
 '/content/output/added_tokens.json',
 '/content/output/tokenizer.json')

In [ ]:
import shutil

# Define the folder path to be zipped
folder_path = "/content/output/deberta_base_2d_5e"

# Define the output zip file path
zip_file_path = "/content/deberta_base_2d_5e.zip"

# Create a zip file from the folder
shutil.make_archive(zip_file_path[:-4], 'zip', folder_path)


'/content/deberta_base_2d_5e.zip'

In [ ]:
del data, original_data, external_data